In [1]:
import torch
import torchvision

In [2]:
import arrayfire as af
import array

In [3]:
def toArrayFire(x):
    x_np = x.detach().contiguous().numpy()
    shape = 1
    if len(x_np.shape) == 0:
        shape = (1,)
    else:
        shape = x_np.shape[::-1]
    afArray = af.Array(x_np.ctypes.data, shape, x_np.dtype.char)
    return afArray

def saveStateDict(module, filepath):
    i = 0
    for (name, param) in module.named_parameters():
        #param = module.state_dict()[name]
        print(name, "\t", param.size())
        if 'in_proj' in name:
            print(param.shape)
            q, k, v = param.chunk(3, dim=0)
            print('in_proj!')
            af.array.save_array(name + 'q', toArrayFire(q), filepath, True)
            af.array.save_array(name + 'k', toArrayFire(k), filepath, True)
            af.array.save_array(name + 'v', toArrayFire(k), filepath, True)
            continue
        if len(param.size()) > 0:
            af_array = toArrayFire(param)
            if 'fc' in name and 'weight' in name:
                af_array = af.array.transpose(af_array)
            af.array.save_array(name, af_array, filepath, True)
            i = i + 1
    print(i)
    for name in module.state_dict():
        if 'running' in name:
            print(name)
            af_array = toArrayFire(module.state_dict()[name])
            af.array.save_array(name, af_array, filepath + 'running', True)

In [4]:
from models.backbone import *
from models.position_encoding import *
from models.matcher import *

In [5]:
from models.transformer import *

In [16]:
batch_size = 1
embedding_size = 128
src_len = 5
tgt_len = 10
num_layers = 2
queries = torch.rand(tgt_len, batch_size, embedding_size)
memory = torch.rand(src_len, batch_size, embedding_size)
layer = TransformerDecoderLayer(embedding_size, 1, dropout=0.0, dim_feedforward=128)
norm = nn.LayerNorm(embedding_size)
model = TransformerDecoder(layer, num_layers, return_intermediate=True, norm=norm)
output = model.forward(queries, memory)

In [17]:
filepath = '/private/home/padentomasello/scratch/pytorch_testing/transformer_decoder.array'
af.array.save_array('queries', toArrayFire(queries), filepath, False)
af.array.save_array('memory', toArrayFire(memory), filepath, True)
i = 2
params = {}
for (name, param) in model.named_parameters():
        if 'in_proj' in name:
            q, k, v = param.chunk(3, dim=0)
            hack = '0'
            if 'in_proj_bias' in name: hack = '1'
            params['0q_' + hack + name] = q
            params['1k_' + hack + name] = k
            params['2v_' + hack + name] = v
            if 'in_proj_bias' in name:
                for key in sorted(params.keys()):
                    af_array = toArrayFire(params[key])
                    if 'weight' in key:
                        af_array = af.array.transpose(af_array)
                    print(key, i, params[key].shape)
                    print(af.array.save_array(key, af_array, filepath, True))
                    i = i + 1
                params = {}
            continue
        elif len(param.size()) > 0:
            af_array = toArrayFire(param)
            if 'fc' in name and 'weight' in name:
                af_array = af.array.transpose(af_array)
            if 'weight' in name and 'proj' in name:
                af_array = af.array.transpose(af_array)
            if 'weight' in name and 'linear' in name:
                af_array = af.array.transpose(af_array)
            print(name, i, param.shape)
            print(af.array.save_array(name, af_array, filepath, True))
            i = i + 1
#af.array.save_array('output', toArrayFire(output), filepath, True)
af.array.save_array('output', toArrayFire(output), filepath, True)

0q_0layers.0.self_attn.in_proj_weight 2 torch.Size([128, 128])
2
0q_1layers.0.self_attn.in_proj_bias 3 torch.Size([128])
3
1k_0layers.0.self_attn.in_proj_weight 4 torch.Size([128, 128])
4
1k_1layers.0.self_attn.in_proj_bias 5 torch.Size([128])
5
2v_0layers.0.self_attn.in_proj_weight 6 torch.Size([128, 128])
6
2v_1layers.0.self_attn.in_proj_bias 7 torch.Size([128])
7
layers.0.self_attn.out_proj.weight 8 torch.Size([128, 128])
8
layers.0.self_attn.out_proj.bias 9 torch.Size([128])
9
0q_0layers.0.multihead_attn.in_proj_weight 10 torch.Size([128, 128])
10
0q_1layers.0.multihead_attn.in_proj_bias 11 torch.Size([128])
11
1k_0layers.0.multihead_attn.in_proj_weight 12 torch.Size([128, 128])
12
1k_1layers.0.multihead_attn.in_proj_bias 13 torch.Size([128])
13
2v_0layers.0.multihead_attn.in_proj_weight 14 torch.Size([128, 128])
14
2v_1layers.0.multihead_attn.in_proj_bias 15 torch.Size([128])
15
layers.0.multihead_attn.out_proj.weight 16 torch.Size([128, 128])
16
layers.0.multihead_attn.out_proj.b

56

In [18]:
output.size()

torch.Size([2, 10, 1, 128])